# Install packages
List of packages that work for the code
*   https://github.com/saeed-anwar/DRLN/issues/12



In [1]:
!pip uninstall --y torchvision
!pip uninstall --y torch
!pip uninstall --y scipy
!pip install -q torch==1.1.0 torchvision==0.3.0
!pip install scipy==1.1.0 # https://github.com/saeed-anwar/DRLN/issues/11

Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Uninstalling torch-1.8.1+cu101:
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling scipy-1.4.1:
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 676.9MB 26kB/s 
     |████████████████████████████████| 2.6MB 56.3MB/s 
ERROR: fastai 1.0.61 requires scipy, which is not installed.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
     |████████████████████████████████| 31.2MB 10.5MB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


Make sure the torch version is 1.1.0

In [2]:
import torch
print(torch.__version__)

1.1.0


# Mount the Google Drive
We can then save the trained model for future usage.

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download code

In [3]:
!git clone https://github.com/Luxi-Zhao/SingleImageSuperResolution.git

Cloning into 'SingleImageSuperResolution'...
remote: Enumerating objects: 1083, done.
remote: Counting objects: 100% (1083/1083), done.
remote: Compressing objects: 100% (798/798), done.
remote: Total 1083 (delta 377), reused 950 (delta 244), pack-reused 0
Receiving objects: 100% (1083/1083), 99.46 MiB | 34.71 MiB/s, done.
Resolving deltas: 100% (377/377), done.


# Download DIV2K

In [15]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

--2021-04-16 21:05:26--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip [following]
--2021-04-16 21:05:27--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 925390592 (883M) [application/zip]
Saving to: ‘DIV2K_train_LR_bicubic_X2.zip’

DIV2K_train_LR_bicu 100%[===================>] 882.52M  20.4MB/s    in 45s     

2021-04-16 21:06:13 (19.4 MB/s) - ‘DIV2K_train_LR_bicubic_X2.zip’ saved [925390592/925390592]

--2021-04-16 21:06:13--  http://data.vision.ee.ethz.ch/cvl/DIV2K

# Unzip the Training Dataset

In [21]:
!mkdir DIV2K
!unzip -q DIV2K_valid_LR_bicubic_X2.zip -d DIV2K
!unzip -q DIV2K_train_LR_bicubic_X2.zip -d DIV2K
!unzip -q DIV2K_train_HR.zip -d DIV2K
!unzip -q DIV2K_valid_HR.zip -d DIV2K

We need to put the validation data into the training data folder because the code expected them to be in the same folder. The code use IDs to distinguish the training set and validation set.

In [22]:
mv DIV2K/DIV2K_valid_HR/* DIV2K/DIV2K_train_HR/

In [25]:
mv DIV2K/DIV2K_valid_LR_bicubic/X2/* DIV2K/DIV2K_train_LR_bicubic/X2/

# Train model
Choose one experiments (0 - 8):

In [39]:
GDRIVE = '/content/drive/MyDrive/2516'
CODE_DIR = '/content/SingleImageSuperResolution/TestCode'
DRLN_experiments = [
               'DRLN_50epoch_Baseline', # Baseline
               'DRLN_50epoch_Ablate_LA_LSC_SSC', # Ablation
               'DRLN_50epoch_Ablate_LA', 
               'DRLN_50epoch_Ablate_LA_LSC', 
               'DRLN_50epoch_Ablate_LSC', 
               'DRLN_50epoch_Ablate_LSC_SSC', 
               'DRLN_50epoch_Ablate_LA_SSC', 
               'DRLN_50epoch_Ablate_SSC',
               'DRLN_50epoch_GFF' # Addition
               ]
model_branch = {
          'DRLN_50epoch_Baseline': 'master',
          'DRLN_50epoch_Ablate_LA_LSC_SSC': 'drln-no-everything',
          'DRLN_50epoch_Ablate_LA': 'drln-no-laplacian',
          'DRLN_50epoch_Ablate_LA_LSC': 'drln-no-long-no-laplacian',
          'DRLN_50epoch_Ablate_LSC': 'drln-no-long-skip',
          'DRLN_50epoch_Ablate_LSC_SSC': 'drln-no-medium-no-long',
          'DRLN_50epoch_Ablate_LA_SSC': 'drln-no-medium-no-laplacian',
          'DRLN_50epoch_Ablate_SSC': 'drln-no-medium-skip',
          'DRLN_50epoch_GFF': 'drln-gff',
}


# Choose One Experiments
experiment = DRLN_experiments[1] # valid [0, 8]
branch = model_branch[experiment]
print("Train with the experiment setting: {}".format(experiment))
print("Branch setting: {}".format(branch))

Train with the experiment setting: DRLN_50epoch_Ablate_LA_LSC_SSC
Branch setting: drln-no-everything


In [27]:
cd /content/SingleImageSuperResolution

/content/SingleImageSuperResolution


In [28]:
!git checkout {branch}

Already on 'drln-no-everything'
Your branch is up to date with 'origin/drln-no-everything'.


Train the model:

In [29]:
cd /content/SingleImageSuperResolution/TestCode/code

/content/SingleImageSuperResolution/TestCode/code


In [30]:
!python main.py --dir_data ../../.. --ext sep_reset --scale 2 --model DRLN --epochs 50 --patch_size 32 --chop --save 'DRLN_epoch50'

Preparing seperated binary files
Preparing seperated binary files
Making model...
Trainable model parameter count:
34430131
Preparing loss function:
1.000 * L1
[Epoch 1]	Learning rate: 1.00e-4
[1600/16000]	[L1: 43.1277]	11.2+0.3s
[3200/16000]	[L1: 33.6684]	11.0+0.0s
[4800/16000]	[L1: 30.3469]	11.0+0.0s
[6400/16000]	[L1: 28.2861]	11.1+0.0s
[8000/16000]	[L1: 26.7633]	11.0+0.0s
[9600/16000]	[L1: 25.7830]	11.0+0.0s
[11200/16000]	[L1: 24.7005]	11.0+0.0s
[12800/16000]	[L1: 23.4642]	11.0+0.0s
[14400/16000]	[L1: 22.3519]	11.0+0.0s
[16000/16000]	[L1: 21.3120]	11.0+0.0s

Evaluation:
100%|███████████████████████████████████████████| 10/10 [01:41<00:00, 10.12s/it]
[DIV2K x2]	PSNR: 26.317 (Best: 26.317 @epoch 1)
[DIV2K x2]	SSIM: 0.564
Total time: 101.25s, ave time: 10.12s

[Epoch 2]	Learning rate: 1.00e-4
[1600/16000]	[L1: 11.2561]	11.1+0.2s
[3200/16000]	[L1: 11.0085]	11.1+0.0s
[4800/16000]	[L1: 10.9534]	11.1+0.0s
[6400/16000]	[L1: 10.8456]	11.1+0.0s
[8000/16000]	[L1: 10.6984]	11.0+0.0s
[9600/16000

## Test Trained Model

Load model from checkpoint

In [32]:
!python main.py --data_test MyImage --scale 2 --model DRLN --n_feats 64 --pre_train /content/SingleImageSuperResolution/TestCode/SR/BI/DRLN_epoch50/model/model_latest.pt --test_only --save_results --chop --save 'DRLN_epoch50' --testpath ../LR/LRBI --testset Set5

Making model...
Loading model from /content/SingleImageSuperResolution/TestCode/SR/BI/DRLN_epoch50/model/model_latest.pt
Trainable model parameter count:
34430131

Evaluation:
100%|█████████████████████████████████████████████| 5/5 [00:02<00:00,  1.82it/s]
[MyImage x2]	PSNR: 27.677 (Best: 27.677 @epoch 1)
[MyImage x2]	SSIM: 0.722
Total time: 2.75s, ave time: 0.55s



# Save Model to Google Drive

In [33]:
mkdir /content/{experiment}

In [34]:
cp -r /content/SingleImageSuperResolution/TestCode/SR/BI/* /content/{experiment}

In [35]:
!zip -r -q {experiment}.zip /content/{experiment}

In [38]:
cp {experiment}.zip /content/drive/MyDrive/2516/